In [5]:
import numpy as np
import torch
import pandas as pd
from transformers import EsmModel, EsmTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

original_seq = 'SWIKEKKLL'
mutated_seq = 'SWIKAKKLL'

model_dir = 'nmodel'  
model = EsmModel.from_pretrained(model_dir).to(device)
tokenizer = EsmTokenizer.from_pretrained(model_dir)

inputs = tokenizer(original_seq, mutated_seq, return_tensors='pt', max_length = 25, padding=True, truncation=True)

inputs = {key: value.to(device) for key, value in inputs.items()}

try:
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.squeeze(0).cpu().numpy()
except Exception as e:
    print(f"Error processing sequence {e}")


print(embeddings)

Some weights of EsmModel were not initialized from the model checkpoint at nmodel and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using device: cpu
[[ 0.01033432  0.42649978  0.29383668 ...  0.8588025   0.07946299
  -0.3298539 ]
 [ 0.07060424 -0.15151554 -0.25000992 ...  0.31884408  0.08618888
   0.02217117]
 [ 0.1535934  -0.29687938  0.11207227 ...  0.02473515  0.1170501
   0.2055141 ]
 ...
 [ 0.10624892 -0.4869262   0.36693028 ...  0.12577724 -0.09223838
   0.30859724]
 [ 0.0927592  -0.4548666   0.37314674 ...  0.10106964 -0.00621835
   0.30683014]
 [ 0.08724934 -0.16967249  0.3516682  ...  0.19683862 -0.31081042
  -0.03457606]]


C:\Users\Rohit VP\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import csv

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attention_weights = nn.Parameter(torch.Tensor(hidden_size, 1))
        nn.init.xavier_uniform_(self.attention_weights)

    def forward(self, lstm_outputs):
        attention_scores = torch.matmul(lstm_outputs, self.attention_weights).squeeze(-1) 
        attention_weights = torch.softmax(attention_scores, dim=1) 
        weighted_sum = torch.bmm(attention_weights.unsqueeze(1), lstm_outputs).squeeze(1) 
        return weighted_sum

class ComplexLSTM(nn.Module):
    def __init__(self):
        super(ComplexLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=320, hidden_size=256, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(input_size=512, hidden_size=128, batch_first=True, bidirectional=True)  
        self.attention = Attention(hidden_size=256)
        self.dropout = nn.Dropout(0.7)
        self.fc1 = nn.Linear(256, 64) 
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = self.attention(x)  
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [7]:
import torch
import numpy as np

model_path = "best_model_nine.pt"  
model = ComplexLSTM()
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu'))) 
model.eval() 
embedding_tensor = torch.tensor(embeddings, dtype=torch.float32) 
if len(embedding_tensor.shape) == 2:  
    embedding_tensor = embedding_tensor.unsqueeze(0)

with torch.no_grad():
    output = model(embedding_tensor) 
    predictions = torch.argmax(output, dim=1)  

print("Predictions:", predictions.item()) 

#Forward:1 and Reverse:0

Predictions: 1


C:\Users\Rohit VP\AppData\Local\Temp\ipykernel_12580\1684004029.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location